# Book: Learning LangChain 
by Mayo Oshin and Nuno Campos

## Chapter 1. LLM Fundamentals with LangChain

In [1]:
# Import libraries
%pip install langchain langchain_openai langchain_community langchain-text-splitters  -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import modules
import os
from dotenv import load_dotenv
from langchain_openai.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.runnables import chain
from pydantic import BaseModel, Field

In [3]:
# Set your OpenAI API key
load_dotenv()
OpenAI.api_key = os.getenv("OPENAI_API_KEY")

# Use the LLMs with LangChain

In [4]:
# Call an OpenAI LLM model (rather than a ChatModel)
llm = OpenAI(model="gpt-3.5-turbo-instruct")

# Invoke the model
prompt = "What is the meaning of life?"
response = llm.invoke(prompt)
print(response)



The meaning of life is a philosophical and existential question that has been pondered by humans for centuries. It refers to the purpose, significance, or ultimate goal of human existence. Different individuals and cultures may have different beliefs and perspectives on the meaning of life, but some common themes include finding happiness, fulfilling one's potential, making a positive impact, and seeking spiritual fulfillment. Ultimately, the meaning of life is a subjective concept and can be interpreted differently by each individual.


# Use the ChatModel with LangChain

HumanMessage
A message sent from the perspective of the human, with the user role.

AIMessage
A message sent from the perspective of the AI the human is interacting with, with the assistant role.

SystemMessage
A message setting the instructions the AI should follow, with the system role.

ChatMessage
A message allowing for arbitrary setting of role.

In [24]:
# Innitialize the chat model
chatmodel = ChatOpenAI(model="gpt-3.5-turbo")

# Create a human message
prompt = [HumanMessage('What is the capital of Japan?')]

# Invoke the chat model
completion = chatmodel.invoke(prompt)
print(completion)
print("\n")
print(completion.content)

content='The capital of Japan is Tokyo.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 14, 'total_tokens': 21, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-767eb490-966f-486e-8838-c973e280b063-0' usage_metadata={'input_tokens': 14, 'output_tokens': 7, 'total_tokens': 21, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}


The capital of Japan is Tokyo.


In [7]:
# Add a system message
system_msg = SystemMessage('You are a helpful assistant that responds to questions with three exclamation marks.')
human_msg = HumanMessage('What is the capital of France?')
completion = chatmodel.invoke([system_msg, human_msg])
print(completion.content)

The capital of France is Paris!!!


# Make LLM prompts reusable

In [25]:
# Create a prompt template
template = PromptTemplate.from_template(
"""Answer the question based on the context below. If the question cannot be answered using the information provided answer with "I don't know".
Context: {context}
Question: {question}
Answer: 
""")

# Invoke the prompt template
context = """
A utopia typically describes an imaginary community or society that possesses highly desirable or near-perfect qualities for its members.[1] It was coined by Sir Thomas More for his 1516 book Utopia, which describes a fictional island society in the New World.
Hypothetical utopias focus on, among other things, equality in categories such as economics, government and justice, with the method and structure of proposed implementation varying according to ideology.[2] Lyman Tower Sargent argues that the nature of a utopia is inherently contradictory because societies are not homogeneous and have desires which conflict and therefore cannot simultaneously be satisfied.
"""
question = "When was Abraham Lincoln born?"
prompt = template.invoke({
    "context": context,
    "question": question
})
print(prompt)
print("\n")
print(prompt.text)

text='Answer the question based on the context below. If the question cannot be answered using the information provided answer with "I don\'t know".\nContext: \nA utopia typically describes an imaginary community or society that possesses highly desirable or near-perfect qualities for its members.[1] It was coined by Sir Thomas More for his 1516 book Utopia, which describes a fictional island society in the New World.\nHypothetical utopias focus on, among other things, equality in categories such as economics, government and justice, with the method and structure of proposed implementation varying according to ideology.[2] Lyman Tower Sargent argues that the nature of a utopia is inherently contradictory because societies are not homogeneous and have desires which conflict and therefore cannot simultaneously be satisfied.\n\nQuestion: When was Abraham Lincoln born?\nAnswer: \n'


Answer the question based on the context below. If the question cannot be answered using the information pr

In [26]:
# Feed the prompt to the LLM
completion = llm.invoke(prompt)
print(completion)

I don't know.


In [27]:
# Build a chat prompt template
template = ChatPromptTemplate.from_messages([
    ('system', 'Answer the question based on the context below. If the question cannot be answered using the information provided answer with "I don\'t know".'),
    ('human', 'Context: {context}'),
    ('human', 'Question: {question}'),
])
prompt = template.invoke({
    "context": context,
    "question": question
})

# Create a chat completion
completion = chatmodel.invoke(prompt)
print(completion.content)

I don't know.


# Produce JSON output

In [28]:
# Create a template using the pydantic basemodel
class AnswerWithJustification(BaseModel):
    '''An answer to the user question along with justification for the answer.'''
    answer: str
    '''The answer to the user's question'''
    justification: str
    '''Justification for the answer'''

# Create a prompt
prompt = "Could people go back in time and change history?"

# Create structured output
# Note: with_structured_output will apply the JSONSchema to the chatmodel output
structured_chatmodel = chatmodel.with_structured_output(AnswerWithJustification)
structured_output = structured_chatmodel.invoke(prompt)
print(structured_output)
print("\n")
print(structured_output.answer)
print("\n")
print(structured_output.justification)

answer='No, people cannot go back in time and change history.' justification='According to current scientific understanding, time travel to the past is not possible. The laws of physics, such as causality and the conservation of energy, make it unlikely that time travel to the past could ever be achieved.'


No, people cannot go back in time and change history.


According to current scientific understanding, time travel to the past is not possible. The laws of physics, such as causality and the conservation of energy, make it unlikely that time travel to the past could ever be achieved.


# Other machine-readable formats with output parsers (CSV, XML)

In [12]:
# Use the CommaSeparatedListOutputParser to parse a comma-separated list of items
parser = CommaSeparatedListOutputParser()
items = parser.invoke("one, two, three, four, five")
print(items)

['one', 'two', 'three', 'four', 'five']


# Using the runnable interface

A common interface with these methods:

invoke
Transforms a single input into an output.

batch
Efficiently transforms multiple inputs into multiple outputs.

stream
Streams output from a single input as it’s produced.

In [13]:
# The invoke interface
completion = chatmodel.invoke('Good morning!')
print(completion.content)

Good morning! How may I assist you today?


In [14]:
# The batch interface
completions = chatmodel.batch(["How are you?", "What is up, bro?"])
print(completions[0].content)
print("\n")
print(completions[1].content)

I'm just a computer program, so I don't have feelings or emotions, but I'm here to help you with anything you need. How can I assist you today?


Not much, just here to assist you with anything you need. How can I help you today?


In [15]:
# The stream interface
for token in chatmodel.stream('See you!'):
    print(token.content)


Good
bye
!
 Take
 care
!



# Combine the different LLM components

Imperative
Call them directly, for example with model.invoke()

Declarative
With LangChain Expression Language (LCEL) using pipe (i.e., |)

In [16]:
# The imperative apporach

# Build a chat prompt template
template = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant.'),
    ('human', '{question}'),
])

# Call a chatmodel
chatmodel = ChatOpenAI()

# Combine the prompt template and the chatmodel in a function
# Note: @chain decorator adds the same Runnable interface for any function you write
@chain
def chatbot(question):
    prompt = template.invoke(question)
    return chatmodel.invoke(prompt)

# Invote the above function with a question
question = "What is the key difference between stemming and lemmatization?"
result = chatbot.invoke(question)
print(result.content)

The key difference between stemming and lemmatization lies in their approach to reducing words to their base or root form. 

Stemming involves cutting off prefixes or suffixes from words to get to the base form, which may not always result in a proper word. For example, the word "running" would be stemmed to "run", but "run" is a valid word on its own.

On the other hand, lemmatization involves looking up words in a language dictionary to find the base or root form, known as the lemma. This results in a valid word that makes sense in the language. Using the same example, "running" would be lemmatized to "run", which is a proper dictionary word.

In summary, stemming is a more basic and heuristic process, while lemmatization is a more sophisticated and accurate method of reducing words to their base form.


In [17]:
# Build a streaming chatbot using the imperative appraoch
@chain
def chatbot(question):
    prompt = template.invoke(question)
    for token in chatmodel.stream(prompt):
        yield token.content

# Run the chatbot
question = "Whis planet is closest to the Earth?"
for part in chatbot.stream(question):
    print(part)


The
 planet
 closest
 to
 Earth
 is
 Venus
.



In [18]:
# Asynchronous function
@chain
async def chatbot(question):
    prompt = await template.ainvoke(question)
    return await chatmodel.ainvoke(prompt)

# Invoke the chatbot asynchronously
result = await chatbot.ainvoke({
    "question": "How to put an elephant into a fridge?"
})
print(result.content)

To put an elephant into a fridge, you would need to follow these steps:

1. Open the fridge door.
2. Move any items that are currently inside the fridge to make space for the elephant.
3. Guide or lead the elephant towards the open fridge door.
4. Gently encourage or assist the elephant in entering the fridge.
5. Once the elephant is inside the fridge, carefully close the door.

It's important to note that this is a hypothetical scenario and not something that should actually be attempted with a real elephant, as it is neither safe nor ethical to put an elephant into a fridge.


In [19]:
# Declarative approach using pipe, i.e., |
chatbot = template | chatmodel
result = chatbot.invoke({
    "question": "Who is the current president of the United States?"
})
print(result.content)

As of my last update, the current president of the United States is Joe Biden.


In [21]:
# Declarative approach with stream
chatbot = template | chatmodel
for part in chatbot.stream({
    "question": "Good evening!"
}):
    print(part.content)


Good
 evening
!
 How
 can
 I
 assist
 you
 today
?



In [23]:
# Declarative approach with asynchonous execution
chatbot = template | chatmodel
result = await chatbot.ainvoke({
    "question": "What is the capital of Australia?",
})
print(result.content)

The capital of Australia is Canberra.
